# Import packages

In [1]:
import pandas as pd
from teradataml import *
import ast
import json
import torch
from teradataml.dataframe.copy_to import copy_to_sql
from transformers.generation import GenerationMixin

from modules.db import TeradataDatabase
from modules.models import( 
    OpusTranslationModelConfig, 
                        OpusTranslationModel, 
                        SentenceEmbeddingConfig, 
                        SentenceEmbeddingModel, 
)
from utils import clean_text, load_embedding_model, unicode_clean, load_translation_model
from constants import (
    CLEANED_TEST_DATA_PATH, TRAIN_VAL_DATA_PATH, CLASS_EMBEDDINGS_PATH, PRODUCT_TEST_EMBEDDINGS_PATH, 
    CLEANED_GPC_PATH, CLEANED_TEST_DATA_PATH, TEST_DATA_PATH, E5_LARGE_INSTRUCT_CONFIG_PATH, 
    OPUS_TRANSLATION_CONFIG_PATH, DATA_PATH, PRODUCT_TRAIN_EMBEDDINGS_PATH, QWEN3_8B_CONFIG_PATH, VALIDATION_DATA_PATH,
    FULL_DATA_SET_DATA_PATH, PRODUCT_FULL_DATASET_EMBEDDINGS_PATH,  CLEANED_FULL_DATASET_DATA_PATH,
    CLASS_EMBEDDINGS_PATH_QWEN, PRODUCT_FULL_DATASET_EMBEDDINGS__QWEN_PATH
)


Logger initialized. All logs will be saved to: c:\Users\ss255385\OneDrive - Teradata Corporation\Desktop\AMuRD-Iteration-7\src\logs\borai_20250820_192939.log


## Connect to database

In [3]:
td_db = TeradataDatabase()
td_db.connect()

### Combine Dataset into 1 file

In [4]:
df_train = pd.read_csv(TRAIN_VAL_DATA_PATH)
df_test = pd.read_csv(TEST_DATA_PATH)
df_valid = pd.read_csv(VALIDATION_DATA_PATH)

In [5]:
full_df = pd.concat([df_train, df_test, df_valid], ignore_index=True)
full_df.to_csv(FULL_DATA_SET_DATA_PATH, index=False)

### Insert Product Names in DB

In [21]:
df = pd.read_csv(FULL_DATA_SET_DATA_PATH)
df.head()

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit
0,مونتانا ذره 400 ج,Vegetables & Fruits,مونتانا,400جم,1,NaN,NaN,NaN,كيس,جم
1,Ahmad Tea Fruit And Herb Selection Herbal Teab...,"Tea, Coffee & Hot Drinks",Ahmad Tea,NaN,20,NaN,NaN,NaN,علبة,NaN
2,Lulu Brown Samoon 1pkt,Bakery,Lulu,NaN,1,NaN,NaN,NaN,عبوة,NaN
3,فلفل رومى بلدى حشو وزن,Vegetables & Fruits,NaN,NaN,1,NaN,NaN,NaN,كيس,NaN
4,كانز,Soft Drinks & Juices,NaN,NaN,1,NaN,NaN,NaN,كانز,NaN


In [22]:
df.rename(columns={"Item_Name": "product_name"}, inplace=True)

In [23]:
df.drop_duplicates(subset=["product_name"], inplace=True)
df.dropna(subset=["product_name"], inplace=True)
df.reset_index(inplace=True)
df.rename(columns={'index': 'id'}, inplace=True)
df = df[["id" , "product_name"]]

In [24]:
df['product_name'] = df['product_name'].apply(unicode_clean)

In [25]:
copy_to_sql(df, "products", "demo_user", if_exists="replace")

### Cleaning the Products

In [27]:
cleaning_query = """
UPDATE demo_user.products
SET product_name = LOWER(
                  TRIM(
                    REGEXP_REPLACE(
                      REGEXP_REPLACE(
                        REGEXP_REPLACE(product_name, '[[:digit:]]+', ''), 
                        '[-_/\\|]', ' '),                              
                      '[[:punct:]]', ' '                              
                    )
                  )
                );
"""

In [28]:
tdf = td_db.execute_query(cleaning_query)

### Translating Products 

In [29]:
tdf = td_db.execute_query("Select * from demo_user.products")
df = pd.DataFrame(tdf)
df

,id,product_name
0,0,مونتانا ذره ج
1,18749,بسمة ملوخية جم
2,1,ahmad tea fruit and herb selection herbal teab...
3,18750,فرزه بطاطس مجمده
4,2,lulu brown samoon pkt
...,...,...
37579,47715,تيفانى بريك ريزو وافر ج
37580,47716,كادبورى شوكولاته كريسبللو ج
37581,47717,جهينه كوكتيل ل
37582,47718,دبل ديرى مشروب بنكهه ليمون مل


In [30]:
model = load_translation_model(OPUS_TRANSLATION_CONFIG_PATH)

In [31]:
products = df["product_name"].tolist()
batch_size = 32
translations = []
for i in range(0, len(products), batch_size):
    batch = products[i:i+batch_size]
    batch_translations = [model.translate(p) for p in batch]
    translations.extend(batch_translations)

In [32]:
df["translated_name"] = translations
df = df[["id", "translated_name"]]

In [33]:
copy_to_sql(df, "products", "demo_user", if_exists="replace")

In [34]:
df.to_csv(DATA_PATH / "cleaned_full_dataset.csv")

### Insert Class Names in DB

In [35]:
df = pd.read_csv(FULL_DATA_SET_DATA_PATH)
df.head()

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit
0,مونتانا ذره 400 ج,Vegetables & Fruits,مونتانا,400جم,1,NaN,NaN,NaN,كيس,جم
1,Ahmad Tea Fruit And Herb Selection Herbal Teab...,"Tea, Coffee & Hot Drinks",Ahmad Tea,NaN,20,NaN,NaN,NaN,علبة,NaN
2,Lulu Brown Samoon 1pkt,Bakery,Lulu,NaN,1,NaN,NaN,NaN,عبوة,NaN
3,فلفل رومى بلدى حشو وزن,Vegetables & Fruits,NaN,NaN,1,NaN,NaN,NaN,كيس,NaN
4,كانز,Soft Drinks & Juices,NaN,NaN,1,NaN,NaN,NaN,كانز,NaN


In [36]:
df_class = df["class"].dropna().unique()
# df = df.reset_index(drop=False) 
# df.rename(columns={'index': 'id'}, inplace=True)
df_classes = pd.DataFrame({"class_name": df_class})

In [37]:
df_classes["id"] = df_classes.index
df_classes = df_classes[["id", "class_name"]]

In [38]:
copy_to_sql(df_classes, "classes", "demo_user", if_exists="replace")

In [ ]:
df_classes

In [40]:
df_actual_class = df[["class"]]
df_actual_class.rename(columns={'class': 'class_name'}, inplace=True)
df_actual_class["product_id"] = df_actual_class.index
df_actual_class = df_actual_class[["product_id", "class_name"]]

In [41]:
copy_to_sql(df_actual_class, "actual_classes", "demo_user", if_exists="replace")

### Cleaning the Class Names

In [42]:
cleaning_query = """
UPDATE demo_user.classes
SET class_name = LOWER(
                  TRIM(
                      REGEXP_REPLACE(
                        REGEXP_REPLACE(class_name, '[^\w\s]', ' '),                                
                      '[-_/\\|]', ' ')                           
                  )
                );

"""

In [43]:
td_db.execute_query(cleaning_query)

[]

In [ ]:
tdf = td_db.execute_query("Select * from demo_user.classes")
df = pd.DataFrame(tdf)
df

In [45]:
cleaning_query = """
UPDATE demo_user.actual_classes
SET class_name = LOWER(
                  TRIM(
                      REGEXP_REPLACE(
                        REGEXP_REPLACE(class_name, '[^\w\s]', ' '),                                
                      '[-_/\\|]', ' ')                           
                  )
                );

"""

In [46]:
td_db.execute_query(cleaning_query)

[]

In [47]:
tdf = td_db.execute_query("Select * from demo_user.actual_classes")
df = pd.DataFrame(tdf)
df

,product_id,class_name
0,0,vegetables fruits
1,49149,rice pasta pulses
2,32766,rice pasta pulses
3,1,tea coffee hot drinks
4,49150,poultry
...,...,...
52489,32761,sauces dressings condiments
52490,32762,biscuits cakes
52491,32763,soft drinks juices
52492,32764,sauces dressings condiments


### Create Product Embeddings

In [4]:
tdf = td_db.execute_query("Select * from demo_user.products")
df = pd.DataFrame(tdf)
df

,id,translated_name
0,0,Montana Corn C
1,40837,Reb i Estek
2,18749,Basma Molokhiya
3,40838,Americana Beans with ghee g
4,1,AHMAD TEA FRUIT AND HERB SELECTION HERBAL TEAB...
...,...,...
37579,40833,Fragile disc.
37580,18745,Marmaris Jam Pasta
37581,40834,Tuna Jam Rose
37582,18747,Tiger kebab potato chips gm


In [5]:
# model = load_falcon3_embedding_model(FALCON3_7B_CONFIG_PATH)
model = load_embedding_model(QWEN3_8B_CONFIG_PATH)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
products = df["translated_name"].tolist()
#embeddings = model.get_embeddings(products)
batch_size = 26
embeddings = []

for i in range(0, len(products), batch_size):
    batch = products[i:i+batch_size]
    batch_embeddings = model.get_embeddings(batch)

    # Ensure each batch is a proper list of lists
    if hasattr(batch_embeddings, "tolist"):
        batch_embeddings = batch_embeddings.tolist()

    embeddings.extend(batch_embeddings)

In [9]:
len(embeddings)

37584

In [10]:
df["embeddings"] = embeddings
len(df["embeddings"][0])

1024

In [11]:
df = df[["id", "embeddings"]]
df.to_csv(PRODUCT_FULL_DATASET_EMBEDDINGS__QWEN_PATH)

### Insert Product Embeddings in DB (Directly from CSV)

In [ ]:
df = pd.read_csv(PRODUCT_FULL_DATASET_EMBEDDINGS__QWEN_PATH)

df["embeddings"] = df["embeddings"].apply(ast.literal_eval)

In [ ]:
emb_cols = pd.DataFrame(df["embeddings"].to_list(), columns=[f"embed_{i}" for i in range(len(df["embeddings"][0]))])

In [ ]:
df_expanded = pd.concat([df[['id']], emb_cols], axis=1)
df_expanded

,id,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,...,embed_1014,embed_1015,embed_1016,embed_1017,embed_1018,embed_1019,embed_1020,embed_1021,embed_1022,embed_1023
0,0,0.015328,0.039254,-0.003427,-0.037174,0.006399,-0.043421,-0.010520,0.065841,0.015048,...,-0.029317,-0.012436,-0.017701,-0.027084,0.021784,0.057254,0.006207,-0.015104,-0.029992,0.046426
1,40837,0.029369,0.050631,0.001408,-0.055559,-0.008562,-0.009442,0.005418,0.018761,0.006816,...,-0.004117,-0.020857,0.000761,-0.010358,0.029784,0.029281,-0.000801,-0.029878,-0.024836,0.045829
2,18749,0.030012,0.014037,0.013602,-0.053966,-0.001614,-0.008398,0.015482,0.014098,0.028291,...,-0.030414,-0.007876,0.017925,-0.008461,0.018188,0.047249,0.018842,-0.011374,-0.035489,0.009465
3,40838,0.036557,0.044264,0.001074,-0.026326,0.020214,-0.029555,-0.007647,0.053449,0.013849,...,-0.033112,-0.020942,0.017518,-0.046515,0.026689,0.037485,-0.020751,-0.029199,-0.041687,0.034662
4,1,0.042021,0.029750,-0.029577,-0.036600,0.030576,-0.050543,-0.027985,0.035200,0.034652,...,-0.052281,-0.079466,0.016898,-0.028278,0.013814,0.037772,0.025222,-0.044246,-0.055822,0.016314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37579,40833,0.025366,0.027520,-0.007039,-0.026696,0.037357,-0.009309,0.000706,0.076981,0.042368,...,-0.024959,-0.027232,0.030310,-0.012734,0.033812,0.036596,0.043399,-0.030106,-0.046717,0.015050
37580,18745,0.031433,0.040032,0.011296,-0.035078,0.006681,0.009141,-0.001431,0.023426,0.014119,...,-0.020247,-0.037832,0.021014,-0.007632,0.030130,0.032414,-0.010519,-0.021812,-0.041907,0.022395
37581,40834,0.029359,0.036676,-0.016532,-0.037347,0.001596,-0.029049,-0.011044,0.032673,0.041653,...,-0.035220,-0.044813,-0.003143,0.009704,0.031135,0.037403,-0.001130,-0.032121,-0.046928,0.012263
37582,18747,0.019306,0.034287,-0.002833,-0.042215,0.011933,-0.015229,-0.016382,0.012368,0.037919,...,-0.046862,-0.024158,0.020916,-0.040049,0.002285,0.004963,0.017478,-0.029646,-0.049897,0.004193


In [ ]:
copy_to_sql(df_expanded, "p_embeddings", "demo_user", if_exists="replace")

### Insert Class Embeddings in DB (Directly from CSV)

In [ ]:
df = pd.read_csv(DATA_PATH / "full_dataset.csv")
df.head()

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit
0,مونتانا ذره 400 ج,Vegetables & Fruits,مونتانا,400جم,1,NaN,NaN,NaN,كيس,جم
1,Ahmad Tea Fruit And Herb Selection Herbal Teab...,"Tea, Coffee & Hot Drinks",Ahmad Tea,NaN,20,NaN,NaN,NaN,علبة,NaN
2,Lulu Brown Samoon 1pkt,Bakery,Lulu,NaN,1,NaN,NaN,NaN,عبوة,NaN
3,فلفل رومى بلدى حشو وزن,Vegetables & Fruits,NaN,NaN,1,NaN,NaN,NaN,كيس,NaN
4,كانز,Soft Drinks & Juices,NaN,NaN,1,NaN,NaN,NaN,كانز,NaN


In [4]:
df_class = df["class"].dropna().unique()
# df = df.reset_index(drop=False) 
# df.rename(columns={'index': 'id'}, inplace=True)
df = pd.DataFrame({"class": df_class})

In [5]:
# model = load_falcon3_embedding_model(FALCON3_7B_CONFIG_PATH)
model = load_embedding_model(E5_LARGE_INSTRUCT_CONFIG_PATH)

In [6]:
classes = df["class"].tolist()
embeddings = model.get_embeddings(classes)
# batch_size = 16
# embeddings = []
# for i in range(0, len(classes), batch_size):
#     batch = classes[i:i+batch_size]
#     batch_translations = model.get_embeddings(batch)
#     embeddings.extend(batch_translations)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
embeddings = embeddings.tolist()
emb_cols = pd.DataFrame(embeddings, columns=[f'embed_{i}' for i in range(len(embeddings[0]))])

In [8]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'id'}, inplace=True)
df_expanded = pd.concat([df[['id']], emb_cols], axis=1)

In [9]:
copy_to_sql(df_expanded, "c_embeddings", "demo_user", if_exists="replace")

### In-DB Similiraity

In [26]:
RESULT_TABLE = "demo_user.similiratiy_score"

In [27]:
# Create the result table
q = f"""
    CREATE TABLE {RESULT_TABLE} (
        item_id BIGINT,
        closest_category_id BIGINT,
        cosine_distance FLOAT
    );
    """

In [28]:
td_db.execute_query(q)

[]

In [29]:
# Generate the string of vector columns for 1024 dimensions
vector_cols = ", ".join([f"embed_{i}" for i in range(1024)])

# Generate the quoted string for feature columns
vector_cols_quoted = ", ".join([f"'embed_{i}'" for i in range(1024)])

In [30]:
classification_sql = f"""
INSERT INTO {RESULT_TABLE}
WITH RankedDistances AS (
    SELECT
        o.Target_ID AS product_id,
        o.Reference_ID AS class_id,
        o.Distance,
        ROW_NUMBER() OVER (PARTITION BY o.Target_ID ORDER BY o.Distance ASC) as rn
    FROM TD_VectorDistance (
        ON (SELECT id, {vector_cols} FROM p_embeddings) AS TargetTable
        ON (SELECT id, {vector_cols} FROM c_embeddings) AS ReferenceTable DIMENSION
        USING
            TargetIDColumn('id')
            RefIDColumn('id')
            TargetFeatureColumns({vector_cols_quoted})
            RefFeatureColumns({vector_cols_quoted})
            DistanceMeasure('cosine')
    ) AS o
)
SELECT
    product_id,
    class_id,
    Distance
FROM RankedDistances
WHERE rn = 1;
"""
td_db.execute_query(classification_sql)

[]

In [31]:
tdf = td_db.execute_query("SELECT * FROM demo_user.similiratiy_score")
tdf = DataFrame(tdf, False)
tdf

item_id,closest_category_id,cosine_distance
0,31,0.13978365945221805
11,10,0.1181549479028765
18,8,0.14703250275497404
1,31,0.1322058824904213
17,11,0.11793198392235371
23,1,0.1251050758861031
10,1,0.1165338996780606
3,2,0.10829480215196019
8,28,0.1563936197276643
2,12,0.13975566093377878


In [32]:
results_query = f"""
SELECT
    p.translated_name AS product_name,
    c.class_name AS predicted_class,
    a.class_name AS actual_class,
    r.cosine_distance AS similarity_score
FROM {RESULT_TABLE} r
JOIN products p
    ON r.item_id = p.id
JOIN classes c
    ON r.closest_category_id = c.id
JOIN actual_classes a
    ON a.product_id = p.id;
"""

In [33]:
tdf = td_db.execute_query(results_query)
df = pd.DataFrame(tdf)

In [34]:
df

,product_name,predicted_class,actual_class,similarity_score
0,Lemon Adalia,jams spreads syrups,tins jars packets,0.132206
1,Candlestick Chocolate Pure Hazelnut,chocolates sweets desserts,biscuits cakes,0.111815
2,Lubya Baladi Jam,jams spreads syrups,rice pasta pulses,0.144473
3,americana okra zero gm,jams spreads syrups,vegetables fruits,0.139784
4,Dasani water ml,water,water,0.108295
...,...,...,...,...
4568,morganic pasta sauce italian bolognese g,rice pasta pulses,tins jars packets,0.144159
4569,Foody fava beans plain k,vegetables fruits,tins jars packets,0.129441
4570,Saw chicken hips.,poultry,poultry,0.160542
4571,Almarai Whole Milk L Plastic,dairy eggs,dairy eggs,0.135945


In [ ]:
df.dropna(inplace=True)

In [36]:
from sklearn.metrics import f1_score

y_pred = df["predicted_class"].tolist()
y_true = df["actual_class"].tolist()

f1_score(y_true, y_pred, average="weighted")

0.4282552067718755

In [37]:
copy_to_sql(df, "results", "demo_user", if_exists="replace")

In [ ]:
query = """
SELECT * FROM TD_ClassificationEvaluator (
   ON demo_user.results AS InputTable
   OUT PERMANENT TABLE OutputTable(classification_metrics)
   USING
       ObservationColumn('actual_class')
       PredictionColumn('predicted_class')
       Labels(
       'condiments dressings marinades', 'furniture', 'personal care skin body care', 'null', 'tea coffee hot Drinks', 'sweets desserts', 'hair shower bath soap', 'fruits', 'nuts dates dried fruits', 'vegetables fruits', 'home appliances', 'sauces dressings condiments', 'baby care', 'tea coffee', 'disposables napkins', 'tins jars packets', 'chips crackers', 'soft drinks juices', 'Cooking Ingredients', 'dairy eggs', 'bakery', 'vegetables herbs', 'biscuits cakes', 'candles air fresheners', 'water', 'rice pasta pulses', 'poultry', 'beef processed Meat', 'home textile', 'cleaning supplies', 'beef lamb meat', 'chocolates sweets desserts', 'jams spreads syrups')
) AS dt;
"""
tdf = td_db.execute_query(query)

In [46]:
DataFrame(td_db.execute_query("select * from demo_user.classification_metrics"))

SeqNum,Metric,MetricValue,index_label
3,Micro-Recall,0.9292452830188679,2
5,Macro-Precision,0.18198704137533162,4
6,Macro-Recall,0.18001895134238396,5
7,Macro-F1,0.16592600446321615,6
9,Weighted-Recall,0.9292452830188679,8
10,Weighted-F1,0.9345127519519589,9
8,Weighted-Precision,0.9565417613453051,7
4,Micro-F1,0.9292452830188679,3
2,Micro-Precision,0.9292452830188679,1
1,Accuracy,0.9292452830188679,0


## Disconnect

In [ ]:
td_db.disconnect()